In [0]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile

In [0]:
!wget --no-check-certificate https://github.com/abhieshekumar/posture-recognition/blob/master/TransferLearning/weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5?raw=true -O /content/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [0]:
local_weights_file = '/content/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = True
  
pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [0]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer
x = layers.Dense(500, activation = 'sigmoid')(x)
x = layers.Dense(204, activation = 'sigmoid')(x)
# Add a final softmax layer
x = layers.Dense  (4, activation='softmax')(x)          

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = Adam(), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

model.summary()

In [0]:
!wget --no-check-certificate -O /content/256.zip https://github.com/abhieshekumar/posture-recognition/blob/master/Dataset/256.zip?raw=true

local_zip = '/content/256.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/content/data/')
zip_ref.close()

# Define our example directories and files
base_dir = '/content/data/'

train_dir = os.path.join( base_dir, 'training')
validation_dir = os.path.join( base_dir, 'validation')

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255. )

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 16 using train_datagen generator
batch_size = 16
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 16 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = batch_size,
                                                          class_mode  = 'categorical', 
                                                          target_size = (150, 150))

In [0]:
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 4000 // batch_size,
            epochs = 40,
            validation_steps = 800 // batch_size,
            verbose = 1)

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [0]:
misc = [1 - x for x in history.history['acc']]
val_misc = [1 - x for x in val_acc]

epochs = range(len(acc))

plt.plot(epochs, misc, 'r', label='Training misclassification')
plt.plot(epochs, val_misc, 'b', label='Validation misclassification')
plt.title('Training and Validation misclassification')
plt.legend(loc=0)
plt.figure()
plt.show()